# Document Scanner

### Importing libraries

In [335]:
from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils

In [336]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    d = np.diff(pts,axis = 1)
    
    rect[1] = pts[np.argmin(d)]
    rect[3] = pts[np.argmax(d)]
    
    
    return rect 

In [337]:
def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl,tr,br,bl) = rect
    
    widthA = np.sqrt(((br[0] - bl[0])**2) + ((br[1] - bl[1])**2))
    widthB = np.sqrt(((tr[0] - tl[0])**2) + ((tr[1] - tl[1])**2))
    
    maxWidth = max(int(widthA), int(widthB))
    
    heightA = np.sqrt(((tr[0] - br[0])**2) + ((tr[1] - br[1])**2))
    heightB = np.sqrt(((tl[0] - bl[0])**2) + ((tl[1] - bl[1])**2))
    
    maxHeight = max(int(heightA), int(heightB))
    
    dst = np.array([[0,0],[maxWidth,0],[maxWidth,maxHeight],[0,maxHeight]], dtype = "float32")
    
    M = cv2.getPerspectiveTransform(rect,dst)
    
    wrapped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    
    return wrapped

### Opening the image

In [338]:
image = cv2.imread('receipt.jpg')

### Making a copy of image

In [339]:
org = image.copy()

### Displaying image

In [315]:
cv2.imshow('img',image)
cv2.waitKey(0)

-1

### Image dimensions

In [340]:
image.shape

(3264, 2448, 3)

### Resizing the image

In [341]:
ratio = image.shape[0] / 500.0     # Taking ratio for use in future

In [358]:
ratio

6.528

In [342]:
image = imutils.resize(image,height=500)
cv2.imshow('img',image)
cv2.waitKey(0)

-1

### Converting image to grayscale

In [343]:
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

### Bluring the image to hide details

In [344]:
blur = cv2.GaussianBlur(gray,(5,5),0)

### Performing edge detection

In [345]:
edges = cv2.Canny(blur,75,200)

In [346]:
cv2.imshow('img',edges)
cv2.waitKey(0)

-1

### Finding contours

In [347]:
cnts = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

In [348]:
cnts = cnts[1]

### Sorting the contours based on its area

In [349]:
# Since the main object in the image will be paper it will have maximum area 
# Taking only the top five with maximum area
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]     

In [350]:
for c in cnts:
    peri = cv2.arcLength(c, True)   # Calculating the perimeter
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)  # finding number of corners
    print(peri, len(approx))
    
    if len(approx) == 4:     # If it has four corners
        screenCnt = approx
        break

1465.8822498321533 4


### Position of the four corners

In [351]:
screenCnt     

array([[[ 27,  35]],

       [[ 18, 462]],

       [[323, 461]],

       [[322,  42]]], dtype=int32)

### Drawing line joining the corners

In [352]:
cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)

-1

### Reshaping the array

In [353]:
screenCnt.reshape(4, 2)

array([[ 27,  35],
       [ 18, 462],
       [323, 461],
       [322,  42]], dtype=int32)

### Applying four point transformation on original image

In [354]:
# The contours are of the resized image so we multiply it by the ratio to obtain contours of original image
warped = four_point_transform(org, screenCnt.reshape(4, 2) * ratio)     

### Converting it to grayscale

In [355]:
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)

### Thresholding 

In [356]:
T = threshold_local(warped, 31, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255

### Displaying the image

In [357]:
cv2.imshow("Outline", imutils.resize(warped,height=600))
cv2.waitKey(0)

-1